# 1. Carregar o dataset e explorar a estrutura do dataframe

In [ ]:
import pandas as pd

# carregar o dataframe, embaralhar os dados, resetar os index e remover coluna "Unnamed" inútil
df_news_70k = (
    pd.read_csv('../dataset/WELFake_Dataset.csv')
    .sample(frac=1)
    .reset_index(drop=True)
    .drop("Unnamed: 0", axis="columns")
)

# mostra as primeiras 5 linhas do df
print("Primeiras 5 linhas do dataframe:")
print(df_news_70k.head())

# mostra a quantidade de linhas e colunas
print(f"\nQuantidade de linhas x colunas: {df_news_70k.shape}")

# 2. Pré processamento: remover nulos, criar amostras menores e preprocessar os textos

In [ ]:
# mostra informações do dataframe
print("Informações sobre os tipos de dados e quantidades de nulos:")
df_news_70k.info()

# apaga as linhas que tem "title" ou "text" nulos, 
df_news_70k.dropna(subset=["title", "text"], inplace=True) # implace ao invés de retornar um novo dataframe ele altera o original direto

print("\nApós apagar os nulos:")
df_news_70k.info()

# verificar distribuição entre notícias reais e falsas

In [ ]:
print("\nContagem de labels após remover nulos  (1 = real, 0 = fake):")
print(df_news_70k['label'].value_counts())

# criar dataframes com tamanho reduzido

In [ ]:
df_news_1k = df_news_70k[:1000].copy()
df_news_10k = df_news_70k[:10000].copy()
df_news_20k = df_news_70k[:20000].copy()
df_news_30k = df_news_70k[:30000].copy()
df_news_40k = df_news_70k[:40000].copy()
df_news_50k = df_news_70k[:50000].copy()

In [ ]:
# escolher o dataframe que será utilizado abaixo
df = df_news_1k.copy()

# pré processamento do texto

passo a passo:
1. Converter o texto para minúsculas
2. Remover pontuação e dígitos
3. Remover palavras irrelevantes como artigos e pronomes(stop words)
4. Aplicar Stemming (remover o sufixo das palavaras exemplo "mudando" "mudaria" para "mud") ou lemmatizing (mais sofisticado reduz a palavra para a forma base "mudaria" ou "mudado" viraria mudar)

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import nltk

nltk.download("punkt") # remove as pontuações
nltk.download("stopwords") # remove as stop words 
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # letras minúsculas
    text = text.lower()

    # remove a pontuação e os digitos
    text = text.translate(str.maketrans('', '', string.punctuation + string.digits))

    # tokeniza o texto
    words = word_tokenize(text)

    # remover as stop words
    words = [word for word in words if word not in stop_words]

    # aplicar lemmatizer nas palavras
    words = [lemmatizer.lemmatize(word) for word in words]
   
    # junta as palavras de volta numa string
    text = ' '.join(words)

    return text

In [ ]:
df['text_clean'] = df['text'].apply(preprocess_text)

# 3. Treinamento do modelo

## **CountVectorizer = BOW (Bag of Words)**

**X** = Matriz onde:
- Cada linha = uma notícia
- Cada coluna = uma palavra do vocabulário total
- Os valores = quantas vezes cada palavra aparece

**y** = Vetor simples com as classificações:
- [1, 0, 1, 0, 1, ...] onde 1 = real, 0 = fake

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_classic = CountVectorizer()  # Cria o "transformador" BOW para os algoritmos clássicos
X_classic = vectorizer_classic.fit_transform(df['text_clean'])  # Aplica BOW nos textos para os algoritmos clássicos

vectorizer_quantum = CountVectorizer(max_features = 12)  # Cria o "transformador" BOW para os algoritmos quânticos
X_quantum = vectorizer_quantum.fit_transform(df['text_clean'])  # Aplica BOW nos textos para os algoritmos quânticos

y = df['label'].values

In [ ]:
from sklearn.model_selection import train_test_split

X_train_classic, X_test_classic, y_train, y_test = train_test_split(X_classic, y, test_size=0.2, random_state=42)

X_train_quantum, X_test_quantum, y_train, y_test = train_test_split(X_quantum, y, test_size=0.2, random_state=42)

# treinar o modelo de regressão linear

In [ ]:
from sklearn.linear_model import LogisticRegression

lcr = LogisticRegression(random_state=42, max_iter=200)
lcr.fit(X_train_classic, y_train)

# treinar o modelo de árvores aleatórias

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(X_train_classic, y_train)

# treinar o modelo com Support Vector Classifier 

In [ ]:
from sklearn.svm import SVC

svm = SVC(kernel='linear', probability=True)
svm.fit(X_train_classic, y_train)

# 4. Avaliação do modelo

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred_lcr = lcr.predict(X_test_classic)
y_pred_rfc = rfc.predict(X_test_classic)
y_pred_svm = svm.predict(X_test_classic)

def evaluate_model(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print('Accuracy:', accuracy)
    print('Precision:', precision)
    print('Recall:', recall)
    print('F1 Score:', f1, "\n")

print("Regressão Logística")
evaluate_model(y_test, y_pred_lcr)

print("Floresta Aleatória")
evaluate_model(y_test, y_pred_rfc)

print("Support Vector Classifier ")
evaluate_model(y_test, y_pred_svm)

# 5. treinar o modelo quântico usando a biblioteca qiskit

In [ ]:
from qiskit.circuit.library import ZZFeatureMap

num_features = X_quantum.shape[1]

feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1)

In [ ]:
from qiskit.circuit.library import RealAmplitudes

ansatz = RealAmplitudes(num_qubits=num_features, reps=3)

In [ ]:
from qiskit_machine_learning.optimizers import COBYLA

optimizer = COBYLA(maxiter=50)

In [ ]:
from qiskit.primitives import StatevectorSampler as Sampler

sampler = Sampler()

In [ ]:
import time
from qiskit_machine_learning.algorithms.classifiers import VQC

vqc = VQC(
    sampler=sampler,
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer,
)

start = time.time()
vqc.fit(X_train_quantum, y_train)
elapsed = time.time() - start

print(f"Training time: {round(elapsed)} seconds")

In [ ]:
# Fazer previsões com o modelo quântico
y_pred_vqc = vqc.predict(X_test_quantum)

# Avaliar o modelo quântico
print("Variational Quantum Classifier")
evaluate_model(y_test, y_pred_vqc)